# <font color="#418FDE" size="6.5" uppercase>**Callable and Descriptor-Oriented Built-ins: callable, classmethod, staticmethod, __import__**</font>

>Last update: 20251208.
    
By the end of this Lecture, you will be able to:
- Use callable to detect whether objects can be invoked in Python 3.12. 
- Define and use classmethod and staticmethod to control how methods receive their first argument. 
- Explain the role of __import__ in the import system and when direct use might be appropriate. 


## **1. Understanding Callability with callable()**

### **1.1. Testing Functions, Methods, and Classes with callable()**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_01_01.jpg?v=1765171788" width="250">



>* callable() checks if an object can run
>* Works for functions, methods, classes, dynamic inputs

>* Functions and methods are callable objects in Python
>* Classes are callable too, constructing and initializing instances

>* callable handles mixed values, functions, and classes
>* focuses on whether objects can be safely invoked



In [ ]:
#@title Python Code - Testing Functions, Methods, and Classes with callable()

# Demonstrate callable with functions, methods, and classes clearly.
# Show how callable helps decide safe object invocation behavior.
# Print results for different objects including functions, methods, and classes.

# Define a simple function that returns a greeting message.
def greet_person(name):
    return f"Hello {name}, welcome aboard today."

# Define a simple class with an instance method and a constructor.
class Greeter:
    def __init__(self, prefix):
        self.prefix = prefix

    def greet(self, name):
        return f"{self.prefix} {name}, nice to meet."

# Create different objects including function, class, instance, and method.
plain_function = greet_person
GreeterClass = Greeter
instance_object = Greeter("Captain")
instance_method = instance_object.greet

# Put several mixed objects into a list for testing callability.
objects_to_test = [plain_function, GreeterClass, instance_object, instance_method, 42]

# Loop through objects, print their type, and whether they are callable.
for obj in objects_to_test:
    print(type(obj).__name__, "callable?", callable(obj))

# Show how callable guides safe usage by conditionally calling or printing directly.
for obj in objects_to_test:
    if callable(obj):
        try:
            print("Result:", obj("Alex"))
        except TypeError:
            print("Result:", obj())
    else:
        print("Non callable value:", obj)



### **1.2. Custom Callables with the __call__ Method**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_01_02.jpg?v=1765171840" width="250">



>* Define __call__ so instances act like functions
>* callable treats functions and custom callable objects alike

>* Custom callables bundle configuration, state, and behavior
>* callable() checks pipeline components share callable interface

>* Custom callables enable pluggable, flexible framework behavior
>* callable() verifies objects meet the invocation contract



In [ ]:
#@title Python Code - Custom Callables with the __call__ Method

# Demonstrate custom callable objects using the __call__ special method.
# Show how instances behave like functions when invoked directly.
# Use callable to verify that objects can actually be invoked safely.

class TemperatureConverter:
    def __init__(self, offset_fahrenheit, label_name):
        self.offset_fahrenheit = offset_fahrenheit
        self.label_name = label_name

    def __call__(self, fahrenheit_value):
        adjusted_value = fahrenheit_value + self.offset_fahrenheit
        celsius_value = (adjusted_value - 32) * 5 / 9
        return f"{self.label_name}: {celsius_value:.1f} degrees Celsius"

# Create two converter instances with different offsets and labels.
converter_morning = TemperatureConverter(offset_fahrenheit=0, label_name="Morning reading")
converter_evening = TemperatureConverter(offset_fahrenheit=5, label_name="Evening adjusted")

# Verify that these objects are callable using the callable built-in.
print("Is converter_morning callable?", callable(converter_morning))
print("Is converter_evening callable?", callable(converter_evening))

# Call the objects like functions to perform temperature conversions.
print(converter_morning(68))
print(converter_evening(68))

# Show that a plain number is not callable and cannot be invoked.
plain_number = 42
print("Is plain_number callable?", callable(plain_number))

# Final demonstration calling both converters again with another Fahrenheit value.
print(converter_morning(77))
print(converter_evening(77))



### **1.3. Callable-Based Validation in Plugin and Callback Architectures**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_01_03.jpg?v=1765171896" width="250">



>* Plugins and callbacks must be actually invokable
>* Use callable() to validate any registered object

>* callable lets many plugin styles work together
>* framework safely invokes any registered callable at runtime

>* Check callability early to catch misconfigurations clearly
>* Scales by enforcing one simple, type-agnostic rule



In [ ]:
#@title Python Code - Callable-Based Validation in Plugin and Callback Architectures

# Demonstrate callable based plugin validation using simple callback examples.
# Show accepting functions, methods, and callable objects as valid callbacks.
# Show rejecting non callable values with clear error messages during registration.

from typing import Any, Callable, List

# Simple registry that stores validated callback plugins.
# We use callable to ensure provided objects are actually invokable.
# This allows flexible plugins like functions, methods, and callable instances.
callback_registry: List[Callable[[int], int]] = []


def register_callback(plugin: Any) -> None:
    # Validate plugin using callable before accepting it into the registry.
    if not callable(plugin):
        print("Rejected plugin, object is not callable:", repr(plugin))
        return
    callback_registry.append(plugin)
    print("Registered plugin successfully:", type(plugin).__name__)


def double_inches(value_inches: int) -> int:
    # Simple function plugin that doubles a length in inches.
    return value_inches * 2


class AdderPlugin:
    # Callable class instance that adds a fixed number of inches.
    def __init__(self, extra_inches: int) -> None:
        self.extra_inches = extra_inches

    def __call__(self, value_inches: int) -> int:
        # Implement call behavior so instances behave like functions.
        return value_inches + self.extra_inches


class MultiplierHolder:
    # Class holding a bound method used as a callback plugin.
    def __init__(self, factor: int) -> None:
        self.factor = factor

    def multiply_inches(self, value_inches: int) -> int:
        # Bound method multiplies the length by the stored factor.
        return value_inches * self.factor


# Create several candidate plugins including invalid non callable values.
valid_function_plugin = double_inches
valid_callable_instance = AdderPlugin(extra_inches=5)
valid_holder = MultiplierHolder(factor=3)
invalid_constant_plugin = 42


# Register different plugin candidates, demonstrating callable based validation.
register_callback(valid_function_plugin)
register_callback(valid_callable_instance)
register_callback(valid_holder.multiply_inches)
register_callback(invalid_constant_plugin)


# Use all successfully registered callbacks on the same base length value.
base_length_inches = 10
print("\nRunning registered callbacks on", base_length_inches, "inches:")
for plugin in callback_registry:
    result = plugin(base_length_inches)
    print("Callback", type(plugin).__name__, "returned", result, "inches")




## **2. Designing Class APIs with classmethod and staticmethod**

### **2.1. Class Methods: Using the Class Itself as self**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_02_01.jpg?v=1765171960" width="250">



>* Class methods receive the class instead of instances
>* Great for alternative constructors and class-wide behavior

>* Class methods see the subclass they’re called on
>* Enable generic constructors and extensible, reusable designs

>* Class methods manage shared class-level state cleanly
>* They support inheritance, separate registries, clearer APIs



In [ ]:
#@title Python Code - Class Methods: Using the Class Itself as self

# Demonstrate class methods receiving the class as first argument.
# Show alternative constructors using class methods for flexible instance creation.
# Compare behavior when calling class methods on base and subclass types.

class TemperatureReading:
    scale_name = "Fahrenheit scale default label"  # Class attribute describing default scale.

    def __init__(self, degrees_fahrenheit):
        self.degrees_fahrenheit = degrees_fahrenheit  # Store temperature in Fahrenheit units.

    @classmethod
    def from_celsius(cls, degrees_celsius):
        degrees_fahrenheit = degrees_celsius * 9 / 5 + 32  # Convert Celsius to Fahrenheit.
        return cls(degrees_fahrenheit)  # Use cls to build correct class instance.

    @classmethod
    def describe_scale(cls):
        return f"Class {cls.__name__} uses {cls.scale_name}."  # Use cls for dynamic name.

    def show(self):
        print(f"Reading {self.degrees_fahrenheit} degrees Fahrenheit value now.")


class PreciseTemperatureReading(TemperatureReading):
    scale_name = "Fahrenheit scale with extra precision"  # Override class attribute description.

    def show(self):
        print(f"Precise reading {self.degrees_fahrenheit:.2f} degrees Fahrenheit value now.")


base_reading = TemperatureReading.from_celsius(25)  # Call class method on base class.
sub_reading = PreciseTemperatureReading.from_celsius(25)  # Call same method on subclass.

print(TemperatureReading.describe_scale())  # Show which class describe_scale sees here.
print(PreciseTemperatureReading.describe_scale())  # Show subclass specific description here.
base_reading.show()  # Display base class reading information.
sub_reading.show()  # Display subclass reading information.




### **2.2. Static Methods: Function-Like Utilities on the Class**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_02_02.jpg?v=1765172018" width="250">



>* Static methods are functions stored on classes
>* Great for domain checks without instance or class

>* Static methods group related helpers on classes
>* Class-qualified calls show helpers belong to domain

>* Static methods group user-related helpers on classes
>* They avoid state access, improving modularity and testing



In [ ]:
#@title Python Code - Static Methods: Function-Like Utilities on the Class

# Demonstrate static methods as function-like utilities on a class.
# Show that static methods receive exactly the arguments we pass.
# Compare instance, class, and static methods for simple temperature helpers.

class TemperatureHelper:
    def __init__(self, degrees_fahrenheit):
        self.degrees_fahrenheit = degrees_fahrenheit

    def to_celsius_instance(self):
        return (self.degrees_fahrenheit - 32) * 5 / 9

    @classmethod
    def from_celsius_class(cls, degrees_celsius):
        fahrenheit_value = degrees_celsius * 9 / 5 + 32
        return cls(fahrenheit_value)

    @staticmethod
    def is_freezing_static(degrees_fahrenheit):
        return degrees_fahrenheit <= 32

    @staticmethod
    def clamp_fahrenheit_static(value, minimum_value, maximum_value):
        return max(minimum_value, min(value, maximum_value))

print("Create helper from celsius using class method.")
helper = TemperatureHelper.from_celsius_class(0)

print("Instance method uses stored fahrenheit value.")
print("Celsius from instance method:", round(helper.to_celsius_instance(), 2))

print("Static method checks arbitrary fahrenheit value.")
print("Is 20F freezing:", TemperatureHelper.is_freezing_static(20))

print("Static method clamps without using instance state.")
print("Clamped 120F between 60F and 100F:", TemperatureHelper.clamp_fahrenheit_static(120, 60, 100))



### **2.3. Practical guidelines for picking instance, class, or static methods**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_02_03.jpg?v=1765172092" width="250">



>* Instance methods need specific object state or data
>* Class methods use class data; static methods utilities

>* Instance methods tie behavior to specific objects
>* Class and static methods support construction and utilities

>* Prefer instance or class methods when truly needed
>* Use static methods sparingly for small helpers



In [ ]:
#@title Python Code - Practical guidelines for picking instance, class, or static methods

# Demonstrate choosing instance, class, and static methods appropriately.
# Show how each method type uses data or ignores it completely.
# Print results to compare method behaviors clearly.

class TemperatureSensor:
    def __init__(self, location_name, offset_fahrenheit):
        self.location_name = location_name
        self.offset_fahrenheit = offset_fahrenheit

    def read_fahrenheit(self, raw_fahrenheit):
        adjusted_value = raw_fahrenheit + self.offset_fahrenheit
        return f"{self.location_name} reading is {adjusted_value} F"

    @classmethod
    def from_celsius_offset(cls, location_name, offset_celsius):
        offset_fahrenheit = offset_celsius * 9 / 5
        return cls(location_name, offset_fahrenheit)

    @staticmethod
    def is_valid_fahrenheit(value_fahrenheit):
        return -100.0 <= value_fahrenheit <= 200.0


# Create instance using direct constructor with explicit Fahrenheit offset.
# This demonstrates behavior depending on specific instance configuration.
indoor_sensor = TemperatureSensor("Living Room", 1.5)
print(indoor_sensor.read_fahrenheit(70.0))

# Create instance using class method with Celsius offset conversion.
# This demonstrates behavior depending on the class itself.
outdoor_sensor = TemperatureSensor.from_celsius_offset("Porch", 0.0)
print(outdoor_sensor.read_fahrenheit(32.0))

# Use static method as simple validation helper utility.
# This demonstrates behavior not needing instance or class context.
print("75 valid?", TemperatureSensor.is_valid_fahrenheit(75.0))

# Show that static method ignores instance and behaves like plain function.
print("300 valid?", outdoor_sensor.is_valid_fahrenheit(300.0))



## **3. Dynamic Imports with __import__ and importlib**

### **3.1. How import Uses __import__ Under the Hood**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_03_01.jpg?v=1765172150" width="250">



>* import statement delegates work to built-in function
>* built-in locates, loads, initializes, and registers modules

>* __import__ finds, loads, and initializes requested modules
>* Loaded modules are cached, avoiding repeated execution

>* Advanced users can call __import__ directly for flexibility
>* Direct use is powerful, low-level, and risky



In [ ]:
#@title Python Code - How import Uses __import__ Under the Hood

# Demonstrate how import statement uses built-in __import__ function internally.
# Show that importing a module twice reuses the same cached module object.
# Compare normal import syntax with calling __import__ directly for clarity.

import math as math_module_alias
import sys as system_module_alias

print("Normal import created math module object once.")
print("math id:", id(math_module_alias))

same_math_again = __import__("math")
print("__import__ returned same math module object.")
print("same_math_again id:", id(same_math_again))

print("Both ids match, showing import uses shared module cache.")
print("math is same_math_again:", math_module_alias is same_math_again)

print("sys.modules contains cached modules including math entry.")
print("'math' in sys.modules:", "math" in system_module_alias.modules)



### **3.2. Dynamic module loading with __import__ in practice**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_03_02.jpg?v=1765172215" width="250">



>* Choose modules at runtime using names as strings
>* Load only needed handlers, keeping application lightweight

>* Platform scans plugin directory and imports modules dynamically
>* Imported plugins registered, enabling decoupled, optional extensions

>* Dynamically choose strategy modules based on conditions
>* Treat module names as data for runtime imports



In [ ]:
#@title Python Code - Dynamic module loading with __import__ in practice

# Demonstrate dynamic module loading using __import__ in a simple plugin style.
# Choose a handler name at runtime and import only that specific module dynamically.
# Show how treating module names as data keeps the core program flexible.

import types as builtin_types_module  # Use builtin types module for creating simple plugin modules.
import sys as builtin_sys_module  # Use builtin sys module for registering our fake plugin modules.

csv_plugin = builtin_types_module.ModuleType("csv_plugin_handler")  # Create csv plugin module object.
json_plugin = builtin_types_module.ModuleType("json_plugin_handler")  # Create json plugin module object.

csv_plugin.handle = lambda data: f"CSV handler processed {data} inches."  # Define csv handler function.
json_plugin.handle = lambda data: f"JSON handler processed {data} inches."  # Define json handler function.

builtin_sys_module.modules["csv_plugin_handler"] = csv_plugin  # Register csv plugin in sys modules.
builtin_sys_module.modules["json_plugin_handler"] = json_plugin  # Register json plugin in sys modules.

available_handlers = ["csv_plugin_handler", "json_plugin_handler"]  # List available handler module names.
print("Available handlers:", available_handlers)  # Show available handler module names to the user.

chosen_name = available_handlers[1]  # Pretend configuration selected the second handler dynamically.
print("Chosen handler name:", chosen_name)  # Show which handler name was selected dynamically.

handler_module = __import__(chosen_name)  # Dynamically import the chosen handler module using __import__.
print("Imported module:", handler_module.__name__)  # Confirm which module object was actually imported.

result = handler_module.handle(12)  # Call the handler function with a simple numeric value.
print("Handler result:", result)  # Display the final result from the dynamically loaded handler.



### **3.3. Prefer importlib Over __import__ for Most Dynamic Imports**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_03_03.jpg?v=1765172280" width="250">



>* Use importlib instead of low-level __import__
>* Importlib makes dynamic imports clearer and maintainable

>* importlib hides changing low-level import details
>* supports robust plugin loading across environments

>* importlib makes dynamic imports clearer and debuggable
>* gives safer control for production plugin loading



In [ ]:
#@title Python Code - Prefer importlib Over __import__ for Most Dynamic Imports

# Demonstrate dynamic imports using importlib instead of low level __import__ directly.
# Show clearer function names and simpler arguments for loading modules dynamically.
# Compare importing math dynamically with importlib and with __import__ built in.

import importlib
import math

# Define a helper function that dynamically imports a module using importlib.
def import_with_importlib(module_name):
    module = importlib.import_module(module_name)
    return module

# Define a helper function that dynamically imports a module using __import__ directly.
def import_with_dunder_import(module_name):
    module = __import__(module_name)
    return module

# Use importlib to import the math module and compute a simple distance in inches.
math_via_importlib = import_with_importlib("math")
leg_a_inches = 36.0
leg_b_inches = 48.0
hypotenuse_inches = math_via_importlib.hypot(leg_a_inches, leg_b_inches)

# Use __import__ directly to import math and compute the same distance in inches.
math_via_dunder = import_with_dunder_import("math")
hypotenuse_inches_dunder = math_via_dunder.hypot(leg_a_inches, leg_b_inches)

# Print results showing that both methods work but importlib reads more clearly overall.
print("Using importlib, hypotenuse inches equals:", hypotenuse_inches)
print("Using __import__, hypotenuse inches equals:", hypotenuse_inches_dunder)
print("Both values match, but importlib call is clearer and safer overall.")




# <font color="#418FDE" size="6.5" uppercase>**Callable and Descriptor-Oriented Built-ins: callable, classmethod, staticmethod, __import__**</font>


In this lecture, you learned to:
- Use callable to detect whether objects can be invoked in Python 3.12. 
- Define and use classmethod and staticmethod to control how methods receive their first argument. 
- Explain the role of __import__ in the import system and when direct use might be appropriate. 

In the next Module (Module 9), we will go over 'Data Conversion, Memory, and Utility Built-ins'